In [ ]:
# installing packages
!pip install transformers langchain langchain_huggingface

In [4]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate

In [5]:
from transformers.utils.logging import set_verbosity_error
set_verbosity_error()

In [6]:
# checking if GPU is being used
import torch
print(torch.cuda.is_available())

True


#### **WORKFLOW**

User Input ->

Summarize (using `facebook/bart-large-cnn` model) ->

Refine (using `google/pegasus-xsum` model) ->

Q/A (using `deepset/roberta-base-squad2` model)

In [10]:
# Summarization
summarization_pipeline = pipeline(
    task = "summarization",
    model = "facebook/bart-large-cnn"
)
summarizer = HuggingFacePipeline(pipeline = summarization_pipeline)

In [12]:
# Refinement
refinement_pipeline = pipeline(
    task = "summarization",
    model = "google/pegasus-xsum"
)
refiner = HuggingFacePipeline(pipeline = refinement_pipeline)

In [ ]:
# Q/A
qa_pipeline = pipeline(
    task = "question-answering",
    model = "deepset/roberta-base-squad2"
)

***INPUT***
(Source: Wikipedia)

Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something becomes useful enough and common enough it's not labeled AI anymore."[2][3]

Various subfields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include learning, reasoning, knowledge representation, planning, natural language processing, perception, and support for robotics.[a] To reach these goals, AI researchers have adapted and integrated a wide range of techniques, including search and mathematical optimization, formal logic, artificial neural networks, and methods based on statistics, operations research, and economics.[b] AI also draws upon psychology, linguistics, philosophy, neuroscience, and other fields.[4] Some companies, such as OpenAI, Google DeepMind and Meta,[5] aim to create artificial general intelligence (AGI)—AI that can complete virtually any cognitive task at least as well as a human.

Artificial intelligence was founded as an academic discipline in 1956,[6] and the field went through multiple cycles of optimism throughout its history,[7][8] followed by periods of disappointment and loss of funding, known as AI winters.[9][10] Funding and interest vastly increased after 2012 when graphics processing units started being used to accelerate neural networks and deep learning outperformed previous AI techniques.[11] This growth accelerated further after 2017 with the transformer architecture.[12] In the 2020s, an ongoing period of rapid progress in advanced generative AI became known as the AI boom. Generative AI's ability to create and modify content has led to several unintended consequences and harms, which has raised ethical concerns about AI's long-term effects and potential existential risks, prompting discussions about regulatory policies to ensure the safety and benefits of the technology.

In [40]:
# taking user input
text_to_summarize = input("Enter the text to summarize:\n")

Enter the text to summarize:
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]  High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applic

In [41]:
# asking how long the summary should be
length = input("Enter the length (short/medium/long):\n")

Enter the length (short/medium/long):
short


In [42]:
# prompt
summary_template = PromptTemplate.from_template("Summarize the following text in {length} way:\n\n{text}")

In [43]:
# chaining
summarization_chain = summary_template | summarizer | refiner

In [44]:
summary = summarization_chain.invoke({
    "text": text_to_summarize,
    "length": length
})

In [45]:
# printing summary
print("Generated Summary:-")
print(summary)

Generated Summary:-
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence.


In [46]:
# Q/A
while True:
  question = input("\nAsk a question (or type 'exit' to stop):\n")
  if question.lower().strip() == 'exit':
    print("Q/A session ended")
    break
  qa_result = qa_pipeline(question=question, context=text_to_summarize)
  print(f"\nANSWER: {qa_result['answer']}\n")


Ask a question (or type 'exit' to stop):
what is artificial intelligence

ANSWER: the capability of computational systems to perform tasks typically associated with human intelligence


Ask a question (or type 'exit' to stop):
what are few high profile applications of AI

ANSWER: robotics


Ask a question (or type 'exit' to stop):
when was artificial intelligence founded

ANSWER: 1956


Ask a question (or type 'exit' to stop):
exit
Q/A session ended
